In [25]:
# --- Celda 1: Imports, rutas, carga y merge con validaciones ---

from pathlib import Path
import pandas as pd
import numpy as np
import sys

# Ajuste de path base según entorno
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(ROOT))

# Funciones personalizadas (asegúrate que existan en utils/etl_utils.py)
from utils.etl_utils import days_diff, pct

# Definición de rutas
BASE = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
RAW = BASE / "data" / "raw"
INTERIM = BASE / "data" / "interim"
PROC = BASE / "data" / "processed"
PROC.mkdir(parents=True, exist_ok=True)

# --- Carga de datos principales ---
parquet_path = INTERIM / "order_items_join_orders.parquet"
assert parquet_path.exists(), f"❌ No existe el archivo intermedio: {parquet_path}"
df = pd.read_parquet(parquet_path)

# --- Carga de datos auxiliares ---
products = pd.read_csv(RAW / "olist_products_dataset.csv", dtype={"product_id": "string"})
sellers = pd.read_csv(RAW / "olist_sellers_dataset.csv", dtype={"seller_id": "string"})

# --- Validaciones previas al merge ---
# Asegurar que no hay IDs duplicados en claves de merge
assert not products["product_id"].duplicated().any(), "❌ Duplicados en 'product_id' en products"
assert not sellers["seller_id"].duplicated().any(), "❌ Duplicados en 'seller_id' en sellers"

# --- Merges ---
df = df.merge(
    products[["product_id", "product_category_name"]],
    on="product_id", how="left", validate="many_to_one"
)

df = df.merge(
    sellers[["seller_id", "seller_city", "seller_state"]],
    on="seller_id", how="left", validate="many_to_one"
)

# --- Verificación final ---
print(f"✅ Dataset cargado correctamente. Shape: {df.shape}")
display(df.head(3))

✅ Dataset cargado correctamente. Shape: (112650, 17)


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_id,product_category_name,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.900002,13.290000,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,3ce436f183e68e07877b285a838db11a,cool_stuff,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.899994,19.930000,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,f6dd3ec061db4e3987629fe6b26e5cce,pet_shop,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.000000,17.870001,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,6489ae5e4333f3693df5ad4372dab6d3,moveis_decoracao,borda da mata,MG


In [26]:
# --- Celda 2: Perfilado rápido de datos ---

# Tipos de datos y conteo de nulos
print("📌 Información general del DataFrame:")
display(df.info())

# Estadísticas descriptivas
print("📊 Estadísticas de columnas numéricas:")
display(df.describe(include=[np.number]))

print("📊 Estadísticas de columnas tipo objeto:")
display(df.describe(include="object"))

print("📊 Estadísticas de columnas categóricas:")
display(df.describe(include="category"))

# --- Nulos: porcentaje y conteo ---
print("🔍 Porcentaje y conteo de valores nulos por columna:")
null_percent = df.isna().mean().round(3) * 100
null_count = df.isna().sum()
nulls_df = pd.DataFrame({
    "nulos (%)": null_percent,
    "nulos (total)": null_count
}).sort_values("nulos (%)", ascending=False)
display(nulls_df)

# --- Duplicados exactos de filas ---
dups = df.duplicated().sum()
print(f"🧯 Duplicados exactos de filas: {dups}")
if dups > 0:
    print("👀 Muestra de duplicados:")
    display(df[df.duplicated()].head())

# --- Duplicados de clave compuesta (order_id + order_item_id) ---
dup_key = df.duplicated(subset=["order_id", "order_item_id"]).sum()
print(f"🔑 Duplicados de clave compuesta (order_id, order_item_id): {dup_key}")
if dup_key > 0:
    display(df[df.duplicated(subset=["order_id", "order_item_id"])].head())

# --- Columnas tipo "object" sin categorizar ---
object_cols = df.select_dtypes(include="object").columns.tolist()
print(f"📝 Columnas tipo 'object' que podrían categorizarse: {object_cols}")

# --- Top categorías de productos ---
print("🏷️ Top 10 categorías de productos:")
display(df["product_category_name"].value_counts(dropna=False).head(10))

# --- Top ciudades de vendedores ---
print("🌍 Top 10 ciudades de vendedores:")
display(df["seller_city"].value_counts(dropna=False).head(10))

📌 Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112650 non-null  string        
 1   order_item_id                  112650 non-null  int16         
 2   product_id                     112650 non-null  string        
 3   seller_id                      112650 non-null  string        
 4   shipping_limit_date            112650 non-null  datetime64[ns]
 5   price                          112650 non-null  float32       
 6   freight_value                  112650 non-null  float32       
 7   order_status                   112650 non-null  category      
 8   order_purchase_timestamp       112650 non-null  datetime64[ns]
 9   order_approved_at              112635 non-null  datetime64[ns]
 10  order_delivered_carrier_date   

None

📊 Estadísticas de columnas numéricas:


,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653732,19.990322
std,0.705124,183.633926,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900002,13.080000
50%,1.000000,74.989998,16.260000
75%,1.000000,134.899994,21.150000
max,21.000000,6735.000000,409.679993


📊 Estadísticas de columnas tipo objeto:


...

,nulos (%),nulos (total)
order_delivered_customer_date,2.2,2454
product_category_name,1.4,1603
order_delivered_carrier_date,1.1,1194
order_id,0.0,0
order_approved_at,0.0,15
seller_city,0.0,0
customer_id,0.0,0
order_estimated_delivery_date,0.0,0
order_purchase_timestamp,0.0,0
order_item_id,0.0,0


🧯 Duplicados exactos de filas: 0
🔑 Duplicados de clave compuesta (order_id, order_item_id): 0
📝 Columnas tipo 'object' que podrían categorizarse: ['product_category_name', 'seller_city', 'seller_state']
🏷️ Top 10 categorías de productos:


product_category_name
cama_mesa_banho           11115
beleza_saude               9670
esporte_lazer              8641
moveis_decoracao           8334
informatica_acessorios     7827
utilidades_domesticas      6964
relogios_presentes         5991
telefonia                  4545
ferramentas_jardim         4347
automotivo                 4235
Name: count, dtype: int64

🌍 Top 10 ciudades de vendedores:


seller_city
sao paulo                27983
ibitinga                  7750
curitiba                  3016
santo andre               2964
belo horizonte            2593
sao jose do rio preto     2579
rio de janeiro            2442
guarulhos                 2362
ribeirao preto            2269
maringa                   2220
Name: count, dtype: int64

In [27]:
# --- Celda 3: Limpieza mínima y ajuste de tipos ---

# 3.1 Eliminar duplicados exactos
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"🧹 Filas eliminadas por duplicado exacto: {before - after}")

# 3.2 Conversión de tipos de datos

# --- IDs como string ---
id_cols = ["order_id", "product_id", "seller_id", "customer_id"]
for c in id_cols:
    if c in df.columns:
        df[c] = df[c].astype("string")
print("🔠 IDs convertidos a string.")

# --- Estado del pedido como categoría ---
if "order_status" in df.columns:
    df["order_status"] = df["order_status"].astype("category")
    print("📦 order_status convertido a category.")

# --- order_item_id como entero compacto ---
if "order_item_id" in df.columns:
    df["order_item_id"] = df["order_item_id"].astype("int16")
    print("🔢 order_item_id convertido a int16.")

# --- Redondear y convertir precios ---
for c in ["price", "freight_value", "total_price"]:
    if c in df.columns:
        df[c] = df[c].round(2).astype("float32")
        print(f"💰 Columna '{c}' redondeada a 2 decimales y convertida a float32.")

# --- Fechas a datetime ---
date_cols = [
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date",
    "order_estimated_delivery_date", "shipping_limit_date"
]
for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")
print("📅 Columnas de fecha convertidas a datetime.")

# --- Categóricos adicionales ---
cat_cols = ["product_category_name", "seller_city", "seller_state"]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")
print("🏷️ Columnas categóricas adicionales convertidas a category.")

# --- Resumen final ---
print("\n✅ Dtypes tras limpieza:")
display(df.dtypes.sort_index())

🧹 Filas eliminadas por duplicado exacto: 0
🔠 IDs convertidos a string.
📦 order_status convertido a category.
🔢 order_item_id convertido a int16.
💰 Columna 'price' redondeada a 2 decimales y convertida a float32.
💰 Columna 'freight_value' redondeada a 2 decimales y convertida a float32.
📅 Columnas de fecha convertidas a datetime.
🏷️ Columnas categóricas adicionales convertidas a category.

✅ Dtypes tras limpieza:


customer_id                      string[python]
freight_value                           float32
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
order_id                         string[python]
order_item_id                             int16
order_purchase_timestamp         datetime64[ns]
order_status                           category
price                                   float32
product_category_name                  category
product_id                       string[python]
seller_city                            category
seller_id                        string[python]
seller_state                           category
shipping_limit_date              datetime64[ns]
dtype: object

In [ ]:
# --- Celda 4: Validación de coherencia temporal ---

from IPython.display import display

print("⏱️ Validando coherencia temporal entre fechas clave...")

# Diccionario para almacenar % de errores
checks = {}
errores = {}

# --- 1. Compra > Aprobación ---
mask_compra_aprobado = (
    df["order_purchase_timestamp"].notna() &
    df["order_approved_at"].notna() &
    (df["order_purchase_timestamp"] > df["order_approved_at"])
)
checks["compra > aprobado"] = pct(mask_compra_aprobado, df)
errores["compra > aprobado"] = df[mask_compra_aprobado]

# --- 2. Aprobación > Despacho ---
mask_aprobado_carrier = (
    df["order_approved_at"].notna() &
    df["order_delivered_carrier_date"].notna() &
    (df["order_approved_at"] > df["order_delivered_carrier_date"])
)
checks["aprobado > carrier"] = pct(mask_aprobado_carrier, df)
errores["aprobado > carrier"] = df[mask_aprobado_carrier]

# --- 3. Despacho > Entrega al cliente ---
mask_carrier_cliente = (
    df["order_delivered_carrier_date"].notna() &
    df["order_delivered_customer_date"].notna() &
    (df["order_delivered_carrier_date"] > df["order_delivered_customer_date"])
)
checks["carrier > cliente"] = pct(mask_carrier_cliente, df)
errores["carrier > cliente"] = df[mask_carrier_cliente]

# --- 4. Compra > Estimación de entrega ---
mask_compra_estimado = (
    df["order_purchase_timestamp"].notna() &
    df["order_estimated_delivery_date"].notna() &
    (df["order_purchase_timestamp"] > df["order_estimated_delivery_date"])
)
checks["compra > estimada"] = pct(mask_compra_estimado, df)
errores["compra > estimada"] = df[mask_compra_estimado]

# --- Mostrar resultados resumidos ---
print("\n📊 Resultados de validación temporal (% de registros con error):")
checks_df = pd.DataFrame.from_dict(checks, orient="index", columns=["% error"]).sort_values(by="% error", ascending=False)
display(checks_df)

# --- Muestra de errores encontrados ---
for name, err_df in errores.items():
    if not err_df.empty:
        print(f"\n🔍 Ejemplos con error: {name} ({len(err_df)} registros)")
        display(err_df[[
            "order_id", "order_purchase_timestamp", "order_approved_at",
            "order_delivered_carrier_date", "order_delivered_customer_date",
            "order_estimated_delivery_date"
        ]].head(3))

⏱️ Validando coherencia temporal entre fechas clave...

📊 Resultados de validación temporal (% de registros con error):


,% error
aprobado > carrier,0.013768
carrier > cliente,0.000444
compra > aprobado,0.000000
compra > estimada,0.000000



🔍 Ejemplos con error: aprobado > carrier (1551 registros)


,order_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
7,000576fe39319847cbb9d288c5617fa6,2018-07-04 12:08:27,2018-07-05 16:35:48,2018-07-05 12:15:00,2018-07-09 14:04:07,2018-07-25
56,002175704e8b209f61b9ad5cfd92b60e,2018-04-22 12:13:25,2018-04-24 17:24:48,2018-04-23 19:03:19,2018-05-02 20:38:44,2018-05-14
67,002834535f7a609a5c68266f173fa59e,2018-07-23 17:26:00,2018-07-28 23:30:59,2018-07-24 15:53:00,2018-08-11 01:48:33,2018-08-14



🔍 Ejemplos con error: carrier > cliente (50 registros)


,order_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
4073,0922ee1619de7b995648e5a8407afb91,2017-07-11 10:45:44,2017-07-11 10:55:15,2017-07-14 12:41:34,2017-07-12 20:49:42,2017-08-14
4074,0922ee1619de7b995648e5a8407afb91,2017-07-11 10:45:44,2017-07-11 10:55:15,2017-07-14 12:41:34,2017-07-12 20:49:42,2017-08-14
11419,19feb5627c41ea1b36a8e50a469b3644,2016-10-07 17:09:56,2016-10-07 17:32:09,2016-10-26 11:42:05,2016-10-20 19:07:54,2016-12-01


In [ ]:
# --- Celda 5: Creación de nuevas variables (features) ---

print("🛠️ Generando variables derivadas...")

# Tiempo real de entrega (en días): desde la compra hasta la entrega al cliente
df["delivery_time_days"] = np.where(
    df["order_delivered_customer_date"].notna(),
    days_diff(df["order_delivered_customer_date"], df["order_purchase_timestamp"]),
    np.nan
)

# Retraso con respecto a la fecha estimada (positivo = llegó tarde)
df["delay_vs_estimated_days"] = np.where(
    df["order_delivered_customer_date"].notna(),
    days_diff(df["order_delivered_customer_date"], df["order_estimated_delivery_date"]),
    np.nan
)

# Conversión a enteros (manteniendo nulos) y redondeo
df["delivery_time_days"] = df["delivery_time_days"].round().astype("Int64")
df["delay_vs_estimated_days"] = df["delay_vs_estimated_days"].round().astype("Int64")

# Variable binaria: llegó tarde (1) o no (0 o NaN)
df["is_late"] = df["delay_vs_estimated_days"].apply(lambda x: 1 if pd.notnull(x) and x > 0 else 0)

# Mostrar ejemplo de variables nuevas
print("\n📊 Variables generadas (muestra):")
display(df[[
    "order_id", "delivery_time_days", "delay_vs_estimated_days", "is_late"
]].head())

Columnas nuevas creadas: ['delivery_time_days', 'delay_vs_estimated_days', 'is_late', 'shipping_days_limit', 'purchase_year', 'purchase_month', 'purchase_day', 'purchase_weekday', 'purchase_hour', 'total_price', 'order_status_simple', 'order_line_uid']
Shape actual: (112650, 29)


In [ ]:
# --- Celda 6: Filtro analítico - Solo pedidos entregados ---

print("📦 Generando subconjunto de pedidos entregados...")

# Copia de seguridad del DataFrame original
df_an = df.copy()

# Filtro: pedidos con estado "delivered" Y fecha de entrega al cliente no nula
mask_delivered = (
    (df_an["order_status"] == "delivered") &
    (df_an["order_delivered_customer_date"].notna())
)
df_an_delivered = df_an.loc[mask_delivered].copy()

# Log de líneas y proporción entregada
total = df.shape[0]
entregados = df_an_delivered.shape[0]
porcentaje = round(100 * entregados / total, 2)

print(f"🔢 Total de líneas originales: {total}")
print(f"📬 Líneas entregadas: {entregados} ({porcentaje}%)")

# Exportar a CSV
entregas_csv_path = PROC / "olist_delivered_only.csv"
df_an_delivered.to_csv(entregas_csv_path, index=False)
print(f"💾 Exportado a: {entregas_csv_path}")

# (Opcional) También exportar a Parquet
# entregas_parquet_path = PROC / "olist_delivered_only.parquet"
# df_an_delivered.to_parquet(entregas_parquet_path, index=False)
# print(f"💾 (Opcional) Exportado también a Parquet: {entregas_parquet_path}")

Total líneas: 112650
Líneas delivered: 110189


In [31]:
# --- celda 7 -- export final ---

# Definir carpeta de salida
DASH = PROC / "dash"
DASH.mkdir(parents=True, exist_ok=True)

# Rutas de archivos
final_csv      = DASH / "olist_final_dataset.csv"
final_xlsx     = DASH / "olist_final_dataset.xlsx"
delivered_csv  = DASH / "olist_delivered_only.csv"

# 🔹 Limitar a 20.000 filas aleatorias del dataset completo
df_sample = df.sample(n=20000, random_state=42)

# 🔹 Limitar a 20.000 filas aleatorias de los pedidos entregados
df_an_delivered_sample = df_an_delivered.sample(n=20000, random_state=42)

# Guardado CSV (separador coma, UTF-8)
df_sample.to_csv(final_csv, index=False)

# Guardado XLSX (útil para Power BI/Excel)
with pd.ExcelWriter(final_xlsx, engine="xlsxwriter") as writer:
    df_sample.to_excel(writer, index=False, sheet_name="data")

# Guardado adicional de subset: pedidos entregados (20.000 filas)
df_an_delivered_sample.to_csv(delivered_csv, index=False)

# 📌 Imprimir conteo final
print("Exportaciones completadas:")
print(f"🟢 Dataset completo → {final_csv.name} ({len(df_sample):,} filas)")
print(f"🟢 Dataset Excel    → {final_xlsx.name} ({len(df_sample):,} filas)")
print(f"🟢 Entregados solo  → {delivered_csv.name} ({len(df_an_delivered_sample):,} filas)")

Exportaciones completadas:
🟢 Dataset completo → olist_final_dataset.csv (20,000 filas)
🟢 Dataset Excel    → olist_final_dataset.xlsx (20,000 filas)
🟢 Entregados solo  → olist_delivered_only.csv (20,000 filas)
